# Pyspark Tutorials

In [4]:
# sc.stop()
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pandas as pd
from pyspark.sql.window import Window
from pyspark.context import SparkContext
from pyspark.sql import SparkSession

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)


In [7]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


df = spark.read.format('csv').option('header', True).option('inferSchema', 'true').load('BigMartSales.csv')

df = df.withColumn('Current_Date', current_date())

df.toPandas().head(5)


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,2025-08-16
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,2025-08-16
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,2025-08-16
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,None,Tier 3,Grocery Store,732.3800,2025-08-16
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,2025-08-16


In [146]:
my_struct_schema = StructType([
    StructField("Item_Identifier", StringType(), True),
    StructField("Item_Weight", DoubleType(), True),
    StructField("Item_Fat_Content", StringType(), True),
    StructField("Item_Visibility", DoubleType(), True),
    StructField("Item_Type", StringType(), True),
    StructField("Item_MRP", DoubleType(), True),
    StructField("Outlet_Identifier", StringType(), True),
    StructField("Outlet_Establishment_Year", IntegerType(), True),
    StructField("Outlet_Size", StringType(), True),
    StructField("Outlet_Location_Type", StringType(), True),
    StructField("Outlet_Type", StringType(), True),
    StructField("Item_Outlet_Sales", DoubleType(), True)
])

In [ ]:
df.select(
    'item_identifier',
    'item_weight', 
    'item_fat_content',
).toPandas().head(5)

,item_identifier,item_weight,item_fat_content
0,FDA15,9.30,Low Fat
1,DRC01,5.92,Regular
2,FDN15,17.50,Low Fat
3,FDX07,19.20,Regular
4,NCD19,8.93,Low Fat


: 

In [80]:
df.select(
    col('item_identifier'),
    col('item_weight'),
    col('item_fat_content')).toPandas().head(5)

,item_identifier,item_weight,item_fat_content
0,FDA15,9.30,Low Fat
1,DRC01,5.92,Regular
2,FDN15,17.50,Low Fat
3,FDX07,19.20,Regular
4,NCD19,8.93,Low Fat


In [81]:
df.select(col('item_identifier').alias('Item ID'),
    col('item_weight').alias('Weight'),
    col('item_fat_content').alias('Fat Content')).toPandas().head(5)

,Item ID,Weight,Fat Content
0,FDA15,9.30,Low Fat
1,DRC01,5.92,Regular
2,FDN15,17.50,Low Fat
3,FDX07,19.20,Regular
4,NCD19,8.93,Low Fat


In [82]:
df.filter(col('item_fat_content') == 'Regular').toPandas().head(5)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date
0,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,2025-08-16
1,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,None,Tier 3,Grocery Store,732.3800,2025-08-16
2,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088,2025-08-16
3,FDO10,13.650,Regular,0.012741,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528,2025-08-16
4,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,None,Tier 2,Supermarket Type1,1076.5986,2025-08-16


In [83]:
df.filter((col('item_type')=='Soft Drinks') & (col('item_weight')<10)).toPandas().head(5)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date
0,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,2025-08-16
1,DRZ11,8.85,Regular,0.113124,Soft Drinks,122.5388,OUT018,2009,Medium,Tier 3,Supermarket Type2,1609.9044,2025-08-16
2,DRF49,7.27,Low Fat,0.071078,Soft Drinks,114.2518,OUT046,1997,Small,Tier 1,Supermarket Type1,2618.5914,2025-08-16
3,DRK12,9.50,LF,0.041878,Soft Drinks,32.9900,OUT035,2004,Small,Tier 2,Supermarket Type1,133.1600,2025-08-16
4,DRK01,7.63,Low Fat,0.061053,Soft Drinks,95.4436,OUT035,2004,Small,Tier 2,Supermarket Type1,1418.1540,2025-08-16


In [84]:
df.filter((col('Outlet_size').isNull() & (col('Outlet_location_type').isin('Tier 1', 'Tier 2')))).toPandas().head(5)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date
0,FDH17,16.20,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,None,Tier 2,Supermarket Type1,1076.5986,2025-08-16
1,FDU28,19.20,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,None,Tier 2,Supermarket Type1,4710.5350,2025-08-16
2,NCD06,13.00,Low Fat,0.099887,Household,45.9060,OUT017,2007,None,Tier 2,Supermarket Type1,838.9080,2025-08-16
3,FDO23,17.85,Low Fat,0.000000,Breads,93.1436,OUT045,2002,None,Tier 2,Supermarket Type1,2174.5028,2025-08-16
4,NCP05,19.60,Low Fat,0.000000,Health and Hygiene,153.3024,OUT045,2002,None,Tier 2,Supermarket Type1,2428.8384,2025-08-16


In [85]:
df.withColumnRenamed('item_identifier', 'ItemID') \
    .withColumnRenamed('item_weight', 'Weight') \
    .withColumnRenamed('item_fat_content', 'FatContent') \
    .toPandas().head(5)

,ItemID,Weight,FatContent,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,2025-08-16
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,2025-08-16
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,2025-08-16
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,None,Tier 3,Grocery Store,732.3800,2025-08-16
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,2025-08-16


In [86]:
df.withColumn('flag', lit('new')).toPandas().head(5)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date,flag
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,2025-08-16,new
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,2025-08-16,new
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,2025-08-16,new
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,None,Tier 3,Grocery Store,732.3800,2025-08-16,new
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,2025-08-16,new


In [87]:
df.withColumn('Product', col('item_weight') * col('item_MRP')).toPandas().head(5)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date,Product
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,2025-08-16,2323.225560
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,2025-08-16,285.753664
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,2025-08-16,2478.315000
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,None,Tier 3,Grocery Store,732.3800,2025-08-16,3496.224000
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,2025-08-16,480.982302


In [88]:
df.withColumn('Item_Fat_Content', regexp_replace(col('item_fat_content'), 'Regular', 'Reg')) \
    .withColumn('Item_Fat_Content', regexp_replace(col('Item_Fat_Content'), 'Low Fat', 'LF')).toPandas().head(5)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date
0,FDA15,9.30,LF,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,2025-08-16
1,DRC01,5.92,Reg,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,2025-08-16
2,FDN15,17.50,LF,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,2025-08-16
3,FDX07,19.20,Reg,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,None,Tier 3,Grocery Store,732.3800,2025-08-16
4,NCD19,8.93,LF,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,2025-08-16


In [89]:
# df.withColumn('Item_weight', col('item_weight').cast(StringType()))

In [90]:
df.printSchema()

root
 |-- Item_Identifier: string (nullable = true)
 |-- Item_Weight: double (nullable = true)
 |-- Item_Fat_Content: string (nullable = true)
 |-- Item_Visibility: double (nullable = true)
 |-- Item_Type: string (nullable = true)
 |-- Item_MRP: double (nullable = true)
 |-- Outlet_Identifier: string (nullable = true)
 |-- Outlet_Establishment_Year: integer (nullable = true)
 |-- Outlet_Size: string (nullable = true)
 |-- Outlet_Location_Type: string (nullable = true)
 |-- Outlet_Type: string (nullable = true)
 |-- Item_Outlet_Sales: double (nullable = true)
 |-- Current_Date: date (nullable = false)



In [91]:
df.sort(col('Item_type').desc()).toPandas().head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date
0,FDH35,18.25,Low Fat,0.000000,Starchy Foods,164.7526,OUT045,2002,None,Tier 2,Supermarket Type1,4604.6728,2025-08-16
1,FDH35,NaN,Low Fat,0.059957,Starchy Foods,165.4526,OUT027,1985,Medium,Tier 3,Supermarket Type3,5262.4832,2025-08-16
2,FDB11,16.00,Low Fat,0.060837,Starchy Foods,226.8404,OUT035,2004,Small,Tier 2,Supermarket Type1,6301.1312,2025-08-16
3,FDB35,12.30,Regular,0.064750,Starchy Foods,92.6804,OUT045,2002,None,Tier 2,Supermarket Type1,1010.6844,2025-08-16
4,FDB35,12.30,Regular,0.064565,Starchy Foods,92.9804,OUT013,1987,High,Tier 3,Supermarket Type1,1929.4884,2025-08-16


In [92]:
df.sort(col('Item_Visibility').asc()).toPandas().head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date
0,FDX07,19.200,Regular,0.0,Fruits and Vegetables,182.0950,OUT010,1998,None,Tier 3,Grocery Store,732.3800,2025-08-16
1,NCD19,8.930,Low Fat,0.0,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,2025-08-16
2,FDP36,10.395,Regular,0.0,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088,2025-08-16
3,FDY07,11.800,Low Fat,0.0,Fruits and Vegetables,45.5402,OUT049,1999,Medium,Tier 1,Supermarket Type1,1516.0266,2025-08-16
4,FDP33,18.700,Low Fat,0.0,Snack Foods,256.6672,OUT018,2009,Medium,Tier 3,Supermarket Type2,3068.0064,2025-08-16


In [93]:
df.sort(['Item_Weight', 'Item_Visibility'], ascending=[0, 0]).toPandas().head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date
0,FDR07,21.35,Low Fat,0.130127,Fruits and Vegetables,96.2094,OUT010,1998,None,Tier 3,Grocery Store,190.4188,2025-08-16
1,FDC02,21.35,Low Fat,0.115195,Canned,258.3278,OUT010,1998,None,Tier 3,Grocery Store,520.6556,2025-08-16
2,FDR07,21.35,Low Fat,0.078061,Fruits and Vegetables,96.0094,OUT018,2009,Medium,Tier 3,Supermarket Type2,380.8376,2025-08-16
3,FDC02,21.35,Low Fat,0.069103,Canned,259.9278,OUT018,2009,Medium,Tier 3,Supermarket Type2,6768.5228,2025-08-16
4,FDC02,21.35,Low Fat,0.068822,Canned,258.3278,OUT046,1997,Small,Tier 1,Supermarket Type1,7028.8506,2025-08-16


In [94]:
df.sort(['Item_Weight', 'Item_Visibility'], ascending=[0, 1]).toPandas().head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date
0,FDC02,21.35,Low Fat,0.068765,Canned,260.4278,OUT013,1987,High,Tier 3,Supermarket Type1,3644.5892,2025-08-16
1,FDC02,21.35,Low Fat,0.068809,Canned,258.5278,OUT035,2004,Small,Tier 2,Supermarket Type1,5206.5560,2025-08-16
2,FDC02,21.35,Low Fat,0.068822,Canned,258.3278,OUT046,1997,Small,Tier 1,Supermarket Type1,7028.8506,2025-08-16
3,FDC02,21.35,Low Fat,0.069103,Canned,259.9278,OUT018,2009,Medium,Tier 3,Supermarket Type2,6768.5228,2025-08-16
4,FDR07,21.35,Low Fat,0.078061,Fruits and Vegetables,96.0094,OUT018,2009,Medium,Tier 3,Supermarket Type2,380.8376,2025-08-16


In [95]:
df.limit(5).toPandas()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,2025-08-16
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,2025-08-16
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,2025-08-16
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,None,Tier 3,Grocery Store,732.3800,2025-08-16
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,2025-08-16


In [96]:
df.drop('Item_Visibility', 'Outlet_Identifier').toPandas().head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Type,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date
0,FDA15,9.30,Low Fat,Dairy,249.8092,1999,Medium,Tier 1,Supermarket Type1,3735.1380,2025-08-16
1,DRC01,5.92,Regular,Soft Drinks,48.2692,2009,Medium,Tier 3,Supermarket Type2,443.4228,2025-08-16
2,FDN15,17.50,Low Fat,Meat,141.6180,1999,Medium,Tier 1,Supermarket Type1,2097.2700,2025-08-16
3,FDX07,19.20,Regular,Fruits and Vegetables,182.0950,1998,None,Tier 3,Grocery Store,732.3800,2025-08-16
4,NCD19,8.93,Low Fat,Household,53.8614,1987,High,Tier 3,Supermarket Type1,994.7052,2025-08-16


In [97]:
df.dropDuplicates().toPandas().head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date
0,FDN04,11.800,reg,0.014087,Frozen Foods,180.3344,OUT046,1997,Small,Tier 1,Supermarket Type1,1427.4752,2025-08-16
1,FDG24,7.975,Low Fat,0.014619,Baking Goods,85.2250,OUT013,1987,High,Tier 3,Supermarket Type1,1081.9250,2025-08-16
2,FDV02,16.750,Low Fat,0.060495,Dairy,169.8106,OUT013,1987,High,Tier 3,Supermarket Type1,3251.1014,2025-08-16
3,FDC40,16.000,Regular,0.065010,Dairy,79.5986,OUT013,1987,High,Tier 3,Supermarket Type1,934.7832,2025-08-16
4,FDP24,20.600,Low Fat,0.082935,Baking Goods,119.5756,OUT013,1987,High,Tier 3,Supermarket Type1,2665.8632,2025-08-16


In [98]:
df.drop_duplicates(subset=['Item_Type']).toPandas()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date
0,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088,2025-08-16
1,FDO23,17.850,Low Fat,0.000000,Breads,93.1436,OUT045,2002,None,Tier 2,Supermarket Type1,2174.5028,2025-08-16
2,FDP49,9.000,Regular,0.069089,Breakfast,56.3614,OUT046,1997,Small,Tier 1,Supermarket Type1,1547.3192,2025-08-16
3,FDC14,NaN,Regular,0.072222,Canned,43.6454,OUT019,1985,Small,Tier 1,Grocery Store,125.8362,2025-08-16
4,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,2025-08-16
5,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,None,Tier 2,Supermarket Type1,1076.5986,2025-08-16
6,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,None,Tier 3,Grocery Store,732.3800,2025-08-16
7,DRI11,NaN,Low Fat,0.034238,Hard Drinks,113.2834,OUT027,1985,Medium,Tier 3,Supermarket Type3,2303.6680,2025-08-16
8,NCB42,11.800,Low Fat,0.008596,Health and Hygiene,115.3492,OUT018,2009,Medium,Tier 3,Supermarket Type2,1621.8888,2025-08-16
9,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,2025-08-16


In [99]:
df.select(col('Item_Type').alias('Item Type')).drop_duplicates(subset=['Item Type']).toPandas()

,Item Type
0,Starchy Foods
1,Baking Goods
2,Breads
3,Fruits and Vegetables
4,Meat
5,Hard Drinks
6,Soft Drinks
7,Household
8,Breakfast
9,Dairy


In [100]:
df.distinct().toPandas().head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date
0,FDN04,11.800,reg,0.014087,Frozen Foods,180.3344,OUT046,1997,Small,Tier 1,Supermarket Type1,1427.4752,2025-08-16
1,FDG24,7.975,Low Fat,0.014619,Baking Goods,85.2250,OUT013,1987,High,Tier 3,Supermarket Type1,1081.9250,2025-08-16
2,FDV02,16.750,Low Fat,0.060495,Dairy,169.8106,OUT013,1987,High,Tier 3,Supermarket Type1,3251.1014,2025-08-16
3,FDC40,16.000,Regular,0.065010,Dairy,79.5986,OUT013,1987,High,Tier 3,Supermarket Type1,934.7832,2025-08-16
4,FDP24,20.600,Low Fat,0.082935,Baking Goods,119.5756,OUT013,1987,High,Tier 3,Supermarket Type1,2665.8632,2025-08-16


In [101]:
data = [('1', 'Kad'),
        ('2', 'Sid')]
Schema1 = 'id STRING, name STRING'

df1 = spark.createDataFrame(data, schema=Schema1)

# df1.toPandas().head()

data2 = [('Omer', '3'),
         ('Ali', '4')]
Schema2 = 'name STRING, id STRING'

df2 = spark.createDataFrame(data2, schema=Schema2)

df1.unionByName(df2).toPandas().head()

,id,name
0,1,Kad
1,2,Sid
2,3,Omer
3,4,Ali


In [102]:
df.select(initcap('Item_Type')).toPandas().head()

,initcap(Item_Type)
0,Dairy
1,Soft Drinks
2,Meat
3,Fruits And Vegetables
4,Household


In [103]:
df.select(lower('Item_Type').alias('Item Type')).toPandas().head()

,Item Type
0,dairy
1,soft drinks
2,meat
3,fruits and vegetables
4,household


In [104]:
df.select(upper('Item_Type').alias('Item Type')).toPandas().head()

,Item Type
0,DAIRY
1,SOFT DRINKS
2,MEAT
3,FRUITS AND VEGETABLES
4,HOUSEHOLD


In [105]:
df.withColumn('curr_date', current_date()).toPandas().head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date,curr_date
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,2025-08-16,2025-08-16
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,2025-08-16,2025-08-16
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,2025-08-16,2025-08-16
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,None,Tier 3,Grocery Store,732.3800,2025-08-16,2025-08-16
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,2025-08-16,2025-08-16


In [106]:
df.withColumn('Week_After', date_add('current_date', 7)).toPandas().head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date,Week_After
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,2025-08-16,2025-08-23
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,2025-08-16,2025-08-23
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,2025-08-16,2025-08-23
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,None,Tier 3,Grocery Store,732.3800,2025-08-16,2025-08-23
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,2025-08-16,2025-08-23


In [107]:
df.withColumn('Week_Before', date_sub('current_date', 7)).toPandas().head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date,Week_Before
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,2025-08-16,2025-08-09
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,2025-08-16,2025-08-09
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,2025-08-16,2025-08-09
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,None,Tier 3,Grocery Store,732.3800,2025-08-16,2025-08-09
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,2025-08-16,2025-08-09


In [108]:
df.withColumn('DateDifference', datediff(dateadd(current_date(), 7), 'current_date')).toPandas().head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date,DateDifference
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,2025-08-16,7
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,2025-08-16,7
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,2025-08-16,7
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,None,Tier 3,Grocery Store,732.3800,2025-08-16,7
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,2025-08-16,7


In [109]:
df.withColumn('Formatted_Current_Date', date_format('Current_date', 'dd-MM-yyyy')).toPandas().head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date,Formatted_Current_Date
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,2025-08-16,16-08-2025
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,2025-08-16,16-08-2025
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,2025-08-16,16-08-2025
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,None,Tier 3,Grocery Store,732.3800,2025-08-16,16-08-2025
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,2025-08-16,16-08-2025


In [110]:
df.dropna('all').toPandas().head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,2025-08-16
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,2025-08-16
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,2025-08-16
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,None,Tier 3,Grocery Store,732.3800,2025-08-16
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,2025-08-16


In [111]:
df.dropna('any').toPandas().head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,2025-08-16
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,2025-08-16
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,2025-08-16
3,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,2025-08-16
4,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088,2025-08-16


In [112]:
df.dropna(subset=['Outlet_Size']).toPandas().head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,2025-08-16
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,2025-08-16
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,2025-08-16
3,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,2025-08-16
4,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088,2025-08-16


In [113]:
df.fillna('NotAvailable').toPandas().head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,2025-08-16
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,2025-08-16
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,2025-08-16
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NotAvailable,Tier 3,Grocery Store,732.3800,2025-08-16
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,2025-08-16


In [114]:
df.fillna('NotAvailable', subset=['Outlet_Size']).toPandas().head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,2025-08-16
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,2025-08-16
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,2025-08-16
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NotAvailable,Tier 3,Grocery Store,732.3800,2025-08-16
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,2025-08-16


In [115]:
df.withColumn('Outlet_Type_Type', split('Outlet_Type', ' ')).toPandas().head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date,Outlet_Type_Type
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,2025-08-16,"[Supermarket, Type1]"
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,2025-08-16,"[Supermarket, Type2]"
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,2025-08-16,"[Supermarket, Type1]"
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,None,Tier 3,Grocery Store,732.3800,2025-08-16,"[Grocery, Store]"
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,2025-08-16,"[Supermarket, Type1]"


In [116]:
df.withColumn('Outlet_Type_Type', split('Outlet_Type', ' ')[1]).toPandas().head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date,Outlet_Type_Type
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,2025-08-16,Type1
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,2025-08-16,Type2
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,2025-08-16,Type1
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,None,Tier 3,Grocery Store,732.3800,2025-08-16,Store
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,2025-08-16,Type1


In [117]:
df.withColumn('Outlet_Type_Type', explode(split('Outlet_Type', ' '))).toPandas().head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date,Outlet_Type_Type
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,2025-08-16,Supermarket
1,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,2025-08-16,Type1
2,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,2025-08-16,Supermarket
3,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,2025-08-16,Type2
4,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,2025-08-16,Supermarket


In [118]:
df.withColumn('Type_Flag', array_contains(split('Outlet_Type', ' '), 'Type1' )).toPandas().head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date,Type_Flag
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,2025-08-16,True
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,2025-08-16,False
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,2025-08-16,True
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,None,Tier 3,Grocery Store,732.3800,2025-08-16,False
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,2025-08-16,True


In [119]:
df.groupBy('Item_Type').agg(sum('Item_MRP')).toPandas().head()

,Item_Type,sum(Item_MRP)
0,Starchy Foods,21880.0274
1,Baking Goods,81894.7364
2,Breads,35379.1198
3,Fruits and Vegetables,178124.0810
4,Meat,59449.8638


In [120]:
df.sort('Item_Type', ascending=False).groupBy('Item_Type').agg(avg('Item_MRP').alias('AVG_MRP')).toPandas().head()

,Item_Type,AVG_MRP
0,Starchy Foods,147.838023
1,Soft Drinks,131.492506
2,Snack Foods,146.194934
3,Seafood,141.841719
4,Others,132.851430


In [121]:
df.groupBy('Item_Type', 'Outlet_Size').agg(sum('Item_MRP').alias('Total_MRP')).toPandas().head()

,Item_Type,Outlet_Size,Total_MRP
0,Starchy Foods,Medium,7124.1362
1,Fruits and Vegetables,Medium,59047.2172
2,Starchy Foods,None,6040.6402
3,Breads,None,10011.5004
4,Baking Goods,None,23433.8388


In [122]:
df.groupBy('Item_Type', 'Outlet_Size').agg(sum('Item_MRP').alias('Total_MRP'), avg('Item_MRP').alias('Average_MRP')).toPandas().head()

,Item_Type,Outlet_Size,Total_MRP,Average_MRP
0,Starchy Foods,Medium,7124.1362,148.419504
1,Fruits and Vegetables,Medium,59047.2172,142.971470
2,Starchy Foods,None,6040.6402,140.480005
3,Breads,None,10011.5004,139.048617
4,Baking Goods,None,23433.8388,126.669399


In [123]:
# Create a sample DataFrame to demonstrate collect_list
sample_data = [
    ("A", "apple"),
    ("A", "apricot"),
    ("B", "banana"),
    ("B", "blueberry"),
    ("C", "cherry")
]
sample_schema = ["Category", "Fruit"]

sample_df = spark.createDataFrame(sample_data, sample_schema)

# Group by 'Category' and collect all 'Fruit' values as a list for each group
sample_df.groupBy('Category').agg(collect_list('Fruit').alias('Fruits')).toPandas()

,Category,Fruits
0,B,"[banana, blueberry]"
1,C,[cherry]
2,A,"[apple, apricot]"


In [124]:
df.groupby('Item_Type').pivot('Outlet_Size').agg(avg('Item_MRP')).toPandas().head()

,Item_Type,null,High,Medium,Small
0,Starchy Foods,140.480005,158.157074,148.419504,150.270174
1,Breads,139.048617,133.758960,140.861039,145.523651
2,Baking Goods,126.669399,129.202044,126.178568,125.213364
3,Fruits and Vegetables,142.575160,145.572870,142.971470,148.313370
4,Meat,139.294534,137.244790,136.419132,145.699250


In [125]:
df.withColumn('VEG/NONVEG_FLAG', when(col('Item_Type')=='MEAT', 'NONVEG').otherwise('VEG')).toPandas().head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date,VEG/NONVEG_FLAG
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,2025-08-16,VEG
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,2025-08-16,VEG
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,2025-08-16,VEG
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,None,Tier 3,Grocery Store,732.3800,2025-08-16,VEG
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,2025-08-16,VEG


In [126]:
df.withColumn('VEG/NONVEG_FLAG',when(col('Item_Type') == 'MEAT', 'NONVEG').otherwise('VEG'))\
        .withColumn('EXP/INEXP_FLAG',when((col('VEG/NONVEG_FLAG') == 'NONVEG') & (col('Item_MRP') > 100), 'NONVEG_EXPENSIVE')
        .when((col('VEG/NONVEG_FLAG') == 'NONVEG') & (col('Item_MRP') <= 100), 'NONVEG_INEXPENSIVE')
        .when((col('VEG/NONVEG_FLAG') == 'VEG') & (col('Item_MRP') > 100), 'VEG_EXPENSIVE')
        .when((col('VEG/NONVEG_FLAG') == 'VEG') & (col('Item_MRP') <= 100), 'VEG_INEXPENSIVE')
        .otherwise('UNKNOWN')).toPandas().head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date,VEG/NONVEG_FLAG,EXP/INEXP_FLAG
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,2025-08-16,VEG,VEG_EXPENSIVE
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,2025-08-16,VEG,VEG_INEXPENSIVE
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,2025-08-16,VEG,VEG_EXPENSIVE
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,None,Tier 3,Grocery Store,732.3800,2025-08-16,VEG,VEG_EXPENSIVE
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,2025-08-16,VEG,VEG_INEXPENSIVE


In [127]:
# Create a Spark session
# spark = SparkSession.builder.appName("InnerJoinExample").getOrCreate()

# Create two sample DataFrames
data1 = [("Alice", 1), ("Bob", 2), ("Cathy", 3)]
data2 = [("Alice", "F"), ("Bob", "M"), ("David", "M")]

df1 = spark.createDataFrame(data1, ["Name", "ID"])
df2 = spark.createDataFrame(data2, ["Name", "Gender"])

# Perform an inner join on the two DataFrames
# joined_df = df1.join(df2, on="Name", how="inner")
df1.join(df2, df1["Name"] == df2["Name"], "inner").toPandas().head()


# Show the result
# joined_df.show()

,Name,ID,Name,Gender
0,Alice,1,Alice,F
1,Bob,2,Bob,M


In [128]:
df1.join(df2, df1['Name']==df2['Name'], 'left').toPandas().head()\
    

,Name,ID,Name,Gender
0,Bob,2,Bob,M
1,Alice,1,Alice,F
2,Cathy,3,None,None


In [129]:
df1.join(df2, df1['Name']==df2['Name'], 'right').toPandas().head()

,Name,ID,Name,Gender
0,Bob,2.0,Bob,M
1,Alice,1.0,Alice,F
2,None,NaN,David,M


In [130]:
df1.join(df2, df1['Name']==df2['Name'], 'anti').toPandas().head()

,Name,ID
0,Cathy,3


In [131]:
df.withColumn('RowNum', row_number().over(Window.orderBy('Item_Identifier'))).toPandas().head() 

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date,RowNum
0,DRA12,11.6,Low Fat,0.041178,Soft Drinks,140.3154,OUT017,2007,None,Tier 2,Supermarket Type1,2552.6772,2025-08-16,1
1,DRA12,11.6,Low Fat,0.000000,Soft Drinks,141.6154,OUT045,2002,None,Tier 2,Supermarket Type1,3829.0158,2025-08-16,2
2,DRA12,11.6,Low Fat,0.040912,Soft Drinks,142.3154,OUT013,1987,High,Tier 3,Supermarket Type1,2552.6772,2025-08-16,3
3,DRA12,11.6,LF,0.000000,Soft Drinks,141.9154,OUT035,2004,Small,Tier 2,Supermarket Type1,992.7078,2025-08-16,4
4,DRA12,11.6,Low Fat,0.041113,Soft Drinks,142.0154,OUT018,2009,Medium,Tier 3,Supermarket Type2,850.8924,2025-08-16,5


In [132]:
df.withColumn('Rnk', rank().over(Window.orderBy('Item_Identifier'))).toPandas().head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date,Rnk
0,DRA12,11.6,Low Fat,0.041178,Soft Drinks,140.3154,OUT017,2007,None,Tier 2,Supermarket Type1,2552.6772,2025-08-16,1
1,DRA12,11.6,Low Fat,0.000000,Soft Drinks,141.6154,OUT045,2002,None,Tier 2,Supermarket Type1,3829.0158,2025-08-16,1
2,DRA12,11.6,Low Fat,0.040912,Soft Drinks,142.3154,OUT013,1987,High,Tier 3,Supermarket Type1,2552.6772,2025-08-16,1
3,DRA12,11.6,LF,0.000000,Soft Drinks,141.9154,OUT035,2004,Small,Tier 2,Supermarket Type1,992.7078,2025-08-16,1
4,DRA12,11.6,Low Fat,0.041113,Soft Drinks,142.0154,OUT018,2009,Medium,Tier 3,Supermarket Type2,850.8924,2025-08-16,1


In [133]:
df.withColumn('Rnk', rank().over(Window.orderBy(col('Item_Identifier').desc())))\
   .withColumn('DenseRnk', dense_rank().over(Window.orderBy(col('Item_Identifier').desc()))).toPandas().head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date,Rnk,DenseRnk
0,NCZ54,14.65,Low Fat,0.000000,Household,161.5552,OUT010,1998,None,Tier 3,Grocery Store,324.9104,2025-08-16,1,1
1,NCZ54,NaN,Low Fat,0.082956,Household,164.0552,OUT027,1985,Medium,Tier 3,Supermarket Type3,5523.4768,2025-08-16,1,1
2,NCZ54,14.65,Low Fat,0.083528,Household,162.4552,OUT045,2002,None,Tier 2,Supermarket Type1,2274.3728,2025-08-16,1,1
3,NCZ54,14.65,Low Fat,0.083359,Household,161.9552,OUT046,1997,Small,Tier 1,Supermarket Type1,4711.2008,2025-08-16,1,1
4,NCZ54,NaN,Low Fat,0.145952,Household,160.9552,OUT019,1985,Small,Tier 1,Grocery Store,162.4552,2025-08-16,1,1


In [134]:
df.withColumn('CumliSum', sum('Item_MRP').over(Window.orderBy('Item_Type'))).toPandas().head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date,CumliSum
0,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088,2025-08-16,81894.7364
1,FDW12,NaN,Regular,0.035400,Baking Goods,144.5444,OUT027,1985,Medium,Tier 3,Supermarket Type3,4064.0432,2025-08-16,81894.7364
2,FDC37,NaN,Low Fat,0.057557,Baking Goods,107.6938,OUT019,1985,Small,Tier 1,Grocery Store,214.3876,2025-08-16,81894.7364
3,FDL12,15.850,Regular,0.121633,Baking Goods,60.6220,OUT046,1997,Small,Tier 1,Supermarket Type1,2576.6460,2025-08-16,81894.7364
4,FDL12,15.850,Regular,0.121532,Baking Goods,59.2220,OUT013,1987,High,Tier 3,Supermarket Type1,599.2200,2025-08-16,81894.7364


In [135]:
df.withColumn('CumliSum', sum('Item_MRP').over(Window.orderBy('Item_Type').rowsBetween(Window.unboundedPreceding, Window.currentRow))).toPandas().head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date,CumliSum
0,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088,2025-08-16,51.4008
1,FDW12,NaN,Regular,0.035400,Baking Goods,144.5444,OUT027,1985,Medium,Tier 3,Supermarket Type3,4064.0432,2025-08-16,195.9452
2,FDC37,NaN,Low Fat,0.057557,Baking Goods,107.6938,OUT019,1985,Small,Tier 1,Grocery Store,214.3876,2025-08-16,303.6390
3,FDL12,15.850,Regular,0.121633,Baking Goods,60.6220,OUT046,1997,Small,Tier 1,Supermarket Type1,2576.6460,2025-08-16,364.2610
4,FDL12,15.850,Regular,0.121532,Baking Goods,59.2220,OUT013,1987,High,Tier 3,Supermarket Type1,599.2200,2025-08-16,423.4830


In [136]:
df.withColumn('TotalSum', sum('Item_MRP').over(Window.orderBy('Item_Type').rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing))).toPandas().head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Current_Date,TotalSum
0,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088,2025-08-16,1.201681e+06
1,FDW12,NaN,Regular,0.035400,Baking Goods,144.5444,OUT027,1985,Medium,Tier 3,Supermarket Type3,4064.0432,2025-08-16,1.201681e+06
2,FDC37,NaN,Low Fat,0.057557,Baking Goods,107.6938,OUT019,1985,Small,Tier 1,Grocery Store,214.3876,2025-08-16,1.201681e+06
3,FDL12,15.850,Regular,0.121633,Baking Goods,60.6220,OUT046,1997,Small,Tier 1,Supermarket Type1,2576.6460,2025-08-16,1.201681e+06
4,FDL12,15.850,Regular,0.121532,Baking Goods,59.2220,OUT013,1987,High,Tier 3,Supermarket Type1,599.2200,2025-08-16,1.201681e+06


In [8]:
df.createTempView('df_view')

In [9]:
spark.sql("SELECT count(*) FROM df_view where Item_Type = 'Dairy' LIMIT 5").show()
# spark.sql('DROP VIEW IF EXISTS df_view')

+--------+
|count(1)|
+--------+
|     682|
+--------+



In [13]:
spark.sql('select distinct Item_Type from df_view').toPandas()

,Item_Type
0,Starchy Foods
1,Baking Goods
2,Breads
3,Fruits and Vegetables
4,Meat
5,Hard Drinks
6,Soft Drinks
7,Household
8,Breakfast
9,Dairy
